In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkConf

sparkConf = SparkConf()
sparkConf.setMaster("spark://spark-master:7077")
sparkConf.setAppName("GCSExample")
sparkConf.set("spark.driver.memory", "2g")
sparkConf.set("spark.executor.cores", "1")
sparkConf.set("spark.driver.cores", "1")
# create the spark session, which is the entry point to Spark SQL engine.
spark = SparkSession.builder.config(conf=sparkConf).getOrCreate()

# Setup hadoop fs configuration for schema gs://
conf = spark.sparkContext._jsc.hadoopConfiguration()
conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
conf.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")

#  Google Storage File Path
gsc_file_path = 'gs://dejads_input/test.csv'  #  use your gcp bucket name. 
# Create data frame load batch data
df = spark.read.format("csv").option("header", "true") \
       .load(gsc_file_path)

df.printSchema()
df.show()

In [ ]:
from pyspark.sql.function import col

df_everyday_count = df.groupBy('date_of_stop', 'subagency').count()

In [4]:
df.write.format("csv").save("gs://dejads_output/lab61.csv") # use correct bucket name

In [6]:
# Stop the spark context
spark.stop()